In [ ]:
pip install pandas numpy scikit-learn surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357237 sha256=3d6a3b9d2d84f615fc53a18e83d5ec2146c32226ab522cdfca8a08dd58df4f9a
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
import pandas as pd

# Load the datasets
movies = pd.read_csv('/content/movies.csv')
ratings = pd.read_csv('/content/ratings.csv')

# Display the first few rows of the datasets
print(movies.head())
print(ratings.head())


   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [ ]:
# Merge movies and ratings dataframes
data = pd.merge(ratings, movies, on='movieId')

# Display the first few rows of the merged dataset
print(data.head())


   userId  movieId  rating   timestamp             title  \
0       1        1     4.0   964982703  Toy Story (1995)   
1       5        1     4.0   847434962  Toy Story (1995)   
2       7        1     4.5  1106635946  Toy Story (1995)   
3      15        1     2.5  1510577970  Toy Story (1995)   
4      17        1     4.5  1305696483  Toy Story (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1  Adventure|Animation|Children|Comedy|Fantasy  
2  Adventure|Animation|Children|Comedy|Fantasy  
3  Adventure|Animation|Children|Comedy|Fantasy  
4  Adventure|Animation|Children|Comedy|Fantasy  


In [ ]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Load data into Surprise format
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Split data into training and test sets
trainset, testset = train_test_split(data, test_size=0.2)

# Build and train the SVD model
algo = SVD()
algo.fit(trainset)

# Make predictions on the test set
predictions = algo.test(testset)

# Compute and print RMSE
accuracy.rmse(predictions)


RMSE: 0.8759


0.8758908441796538

In [ ]:
def get_top_n_recommendations(user_id, n=10):
    # Get a list of all movie IDs
    movie_ids = movies['movieId'].unique()

    # Get a list of movie IDs the user has rated
    user_rated_movie_ids = ratings[ratings['userId'] == user_id]['movieId']

    # Predict ratings for all movies the user hasn't rated yet
    movie_ids_to_predict = [movie_id for movie_id in movie_ids if movie_id not in user_rated_movie_ids]
    predictions = [algo.predict(user_id, movie_id) for movie_id in movie_ids_to_predict]

    # Sort predictions by estimated rating in descending order
    predictions.sort(key=lambda x: x.est, reverse=True)

    # Get the top N predictions
    top_n_predictions = predictions[:n]

    # Get the movie titles for the top N predictions
    top_n_movie_ids = [pred.iid for pred in top_n_predictions]
    top_n_movie_titles = movies[movies['movieId'].isin(top_n_movie_ids)]['title']

    return top_n_movie_titles

# Example: Get top 10 recommendations for user with userId 1
top_recommendations = get_top_n_recommendations(user_id=1, n=10)
print(top_recommendations)


474                                  Blade Runner (1982)
602    Dr. Strangelove or: How I Learned to Stop Worr...
686                                   Rear Window (1954)
690                            North by Northwest (1959)
694                                    Casablanca (1942)
828                                Reservoir Dogs (1992)
863               Monty Python and the Holy Grail (1975)
883                                Paths of Glory (1957)
898    Star Wars: Episode V - The Empire Strikes Back...
900    Raiders of the Lost Ark (Indiana Jones and the...
Name: title, dtype: object


In [ ]:
from surprise.model_selection import GridSearchCV

# Define parameter grid for GridSearchCV
param_grid = {'n_epochs': [20, 30], 'lr_all': [0.005, 0.010], 'reg_all': [0.02, 0.05]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)

# Perform the grid search
gs.fit(data)

# Best RMSE score
print(gs.best_score['rmse'])

# Best parameters
print(gs.best_params['rmse'])

# Use the best model
algo = gs.best_estimator['rmse']
algo.fit(trainset)


0.8679097355398592
{'n_epochs': 30, 'lr_all': 0.01, 'reg_all': 0.05}


In [ ]:
!pip install flask-ngrok
!pip install flask
!pip install pyngrok


In [ ]:
from pyngrok import ngrok

# Insert your Ngrok authtoken here
ngrok_authtoken = '2hEWZvwLigfXrieDWKoxpcykWjt_3gLm3Uc6LCaBEwFgW5Qwh'
ngrok.set_auth_token(ngrok_authtoken)


In [ ]:
from flask import Flask, request, jsonify
from surprise import Dataset, Reader, SVD
import pandas as pd

# Initialize the Flask application
app = Flask(__name__)

# Load your pre-trained model and data
reader = Reader(rating_scale=(0.5, 5.0))
ratings = pd.read_csv('/content/ratings.csv')
movies = pd.read_csv('/content/movies.csv')
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD()
trainset = data.build_full_trainset()
algo.fit(trainset)

@app.route('/')
def home():
    return "Movie Recommendation System is Running"

@app.route('/recommend', methods=['POST'])
def recommend():
    print("Request received!")
    user_id = int(request.json.get('user_id'))
    n = int(request.json.get('n', 10))
    print(f"User ID: {user_id}, Number of recommendations: {n}")

    # Get recommendations
    top_n_recommendations = get_top_n_recommendations(user_id, n)
    print(f"Recommendations: {top_n_recommendations.tolist()}")

    return jsonify({'recommendations': top_n_recommendations.tolist()})


def get_top_n_recommendations(user_id, n=10):
    movie_ids = movies['movieId'].unique()
    user_rated_movie_ids = ratings[ratings['userId'] == user_id]['movieId']
    movie_ids_to_predict = [movie_id for movie_id in movie_ids if movie_id not in user_rated_movie_ids]
    predictions = [algo.predict(user_id, movie_id) for movie_id in movie_ids_to_predict]
    predictions.sort(key=lambda x: x.est, reverse=True)
    top_n_predictions = predictions[:n]
    top_n_movie_ids = [pred.iid for pred in top_n_predictions]
    top_n_movie_titles = movies[movies['movieId'].isin(top_n_movie_ids)]['title']
    return top_n_movie_titles


In [ ]:
# Start Ngrok
public_url = ngrok.connect(5000)
print(f"Ngrok Tunnel URL: {public_url}")

# Run Flask app
app.run()


Ngrok Tunnel URL: NgrokTunnel: "https://5763-35-234-10-140.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [31/May/2024 13:07:04] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/May/2024 13:07:05] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [31/May/2024 13:08:33] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/May/2024 13:09:02] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/May/2024 13:09:03] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [31/May/2024 13:21:20] "POST /recommend HTTP/1.1" 200 -


Request received!
User ID: 1, Number of recommendations: 10
Recommendations: ['Hoop Dreams (1994)', 'Shawshank Redemption, The (1994)', 'In the Name of the Father (1993)', "Schindler's List (1993)", 'Blade Runner (1982)', 'Terminator 2: Judgment Day (1991)', 'Wallace & Gromit: The Best of Aardman Animation (1996)', 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)', 'Godfather, The (1972)', 'Rear Window (1954)']


INFO:werkzeug:127.0.0.1 - - [31/May/2024 13:22:41] "GET / HTTP/1.1" 200 -
